In [1]:
from pathlib import Path
import pandas as pd
import os
import gzip
import json
from utils import extract_gz, load_and_process_data

## Dataset

In [2]:
dataset_path = Path(
    "~/Datasets/Amazon_Reviews_23/meta_data/meta_Clothing_Shoes_and_Jewelry.jsonl.gz"
).expanduser()
assert dataset_path.exists(), "There is no dataset file!"

Unpack the data

In [11]:
extract_path = extract_gz(path=dataset_path)
print(f"File extracted to: {extract_path}")

File extracted to: /Users/studeni/Datasets/Amazon_Reviews_23/meta_data/meta_Clothing_Shoes_and_Jewelry.jsonl


In [3]:
extract_path = Path(
    "/Users/studeni/Datasets/Amazon_Reviews_23/meta_data/meta_Clothing_Shoes_and_Jewelry.jsonl"
)

In [4]:
extract_path.exists()

True

Load JSONL to DataFrame

In [13]:
df = load_and_process_data(file_path=extract_path)